### Import and Organise the data####

In [21]:
from sklearn.datasets import fetch_mldata
import numpy as np
from sklearn.utils import shuffle
mnist = fetch_mldata('MNIST original', data_home=".")
mnist.data, mnist.target = shuffle(mnist.data, mnist.target)
X = mnist.data
y_float = mnist.target
y_int = y_float.astype(np.int64)
y = np.eye(10)[y_int]

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
s_train = np.ones(X_train.shape[0])
X_train = np.hstack((X_train, s_train.reshape(len(s_train),1)))
s_test = np.ones(X_test.shape[0])
X_test = np.hstack((X_test, s_test.reshape(len(s_test),1)))

## define useful functions 

In [23]:
def sigmoid(a):
    return 1.0 / (1.0 + np.exp(-a))


### gradient descent

In [24]:
def gradient(x, y, w1, w2, w3, theta1, theta2,theta3):
    z1 = np.matmul(x, w1)
    z2 = np.matmul(x, w2)
    z3 = np.matmul(x, w3)
    zf = np.matmul(z1, theta1) + np.matmul(z2, theta2)+np.matmul(z3, theta3)
    sig = (sigmoid(zf)-y) # [N, 10]
    gradient_c_theta1 = np.matmul(np.transpose(z1), sig) / N #[S1, N] [N, 10]
    gradient_c_theta2 = np.matmul(np.transpose(z2), sig) / N #[S2, N] [N, 10]
    gradient_c_theta3 = np.matmul(np.transpose(z3), sig) / N #[S3,N] [N,10]
    gradient_c_z1 = np.matmul(theta1, np.transpose(sig))  / N #[S1, 10] [N, 10]
    gradient_c_z2 = np.matmul(theta2, np.transpose(sig)) / N 
    gradient_c_z3 = np.matmul(theta3, np.transpose(sig)) / N
    #gradient_z1_w1= np.tranpose(x).T[:,:,None]*np.transpose(np.multiply(sigmoid(z2), (1-sig(z2)))).T[:,None] #[N, S1, 780]
    gradient_z1_w1 =x
    gradient_c_w1 = np.matmul(np.transpose(x),np.transpose(gradient_c_z1))
    gradient_z2_w2=np.ones((N,S2_logistic,785))
    gradient_c_w2=np.ones((S2_logistic,785))
    for i in range(S2_logistic):
        siggy = np.multiply(sigmoid(z2),(1-sigmoid(z2))) ### [N,S2 ]
        gradient_z2_w2[:,i,:] = x * (siggy[:,i].reshape(N,1))   ###[N,S2,785]
    for i in range(S2_logistic):    
        gradient_c_w2[i,:] =np.matmul(gradient_c_z1[i,:],gradient_z2_w2[:,i,:])
    gradient_z3_w3=np.ones((N,S3_Gauss,785))
    gradient_c_w3=np.ones((S3_Gauss,785))
    for i in range(S3_Gauss):
        gradient_z3_w3[:,i,:] = np.multiply((beta*(x-w3[:,i])), np.exp( beta*(np.linalg.norm(x-w3[:,i]))))   #[N, i ,785]
    for i in range(S3_Gauss):
        gradient_c_w3[i,:] =np.matmul(gradient_c_z1[i,:],gradient_z2_w2[:,i,:])
        
    return zf, gradient_c_theta1, gradient_c_theta2, gradient_c_theta3, gradient_c_w1, gradient_c_w2, gradient_c_w3

In [25]:
# z1 = linear_output_z1(X_train, int_w1)
# z2 = logistic_output_z2(X_train, int_w2)
# zf = logistic_output(X_train, z1, z2, int_theta1, int_theta2)

In [26]:
def gradient_descent(x, y, int_w1, int_w2, int_w3, int_theta1, int_theta2, int_theta3, lowtrain, uptrain, iterations,N):

    place_w1=int_w1
    place_w2=int_w2
    place_w3=int_w3
    place_theta1=int_theta1
    place_theta2=int_theta2
    place_theta3=int_theta3

    for i in range(iterations):
        trainrate=np.random.uniform(lowtrain,uptrain,1)/np.cbrt(i+1) 

        zf, gradient_theta1, gradient_theta2, gradient_theta3, gradient_w1, gradient_w2, gradient_w3 = gradient(x, y, place_w1, place_w2, place_w3, place_theta1, place_theta2, place_theta3)
        p = sigmoid(zf)
            
        place_w1 = place_w1 - (trainrate * gradient_w1) - (0.1/N)*place_w1 * trainrate
        place_w2 = place_w2 - (trainrate * np.transpose(gradient_w2)) - (0.1/N)*place_w2 * trainrate
        place_w3 = place_w3 - (trainrate * np.transpose(gradient_w3)) - (0.1/N)*place_w3 * trainrate

        place_theta1 = place_theta1 - trainrate*gradient_theta1*0.1 - (0.1/N)*place_theta1 * trainrate 
        place_theta2 = place_theta2 - trainrate*gradient_theta2 - (0.1/N)*place_theta2 * trainrate
        place_theta3 = place_theta3 - trainrate*gradient_theta3 - (0.1/N)*place_theta3 * trainrate

    return  p, place_theta1, place_theta2, place_theta3, place_w1, place_w2, place_w3

In [27]:
def predict_accuracy(xtest, ytest, theta1, theta2, theta3, w1, w2, w3):
    z1 = np.matmul(xtest, w1)
    z2 = np.matmul(xtest, w2)
    z3 = np.matmul(xtest, w3) 
    zf = np.matmul(z1, theta1) + np.matmul(z2, theta2) + np.matmul(z3, theta3)
    p = sigmoid(zf)
    prediction = np.argmax(p, axis=1)
    answer = np.argmax(ytest, axis=1)
    success_rate = answer == prediction
    success_rate = success_rate*1
    prediction_rate = sum(success_rate) / xtest.shape[0]
    
    return prediction_rate

In [40]:
S1_linear = 3
S2_logistic = 3
S3_Gauss = 3
N=52500 # Batch size
beta = 0.1

int_w1, int_w2,int_w3 = np.random.uniform(-1,1,(X_train.shape[1],S1_linear)), np.random.uniform(-1,1,(X_train.shape[1], S2_logistic)), np.random.uniform(-1,1,(X_train.shape[1], S3_Gauss))
int_theta1, int_theta2, int_theta3 = np.random.uniform(-1,1,(S1_linear,10)), np.random.uniform(-1,1,(S2_logistic,10)), np.random.uniform(-1,1,(S3_Gauss,10))
result = gradient_descent(X_train, y_train, int_w1, int_w2, int_w3, int_theta1, int_theta2, int_theta3, 0.00001, 0.0001, 1,N)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: overflow encountered in exp


In [41]:
for i in range(200):
    X_train, y_train = shuffle(X_train, y_train)
    X_train_mini, y_train_mini = X_train[0:200], y_train[0:200]
    N = X_train_mini.shape[0]
    w1 = result[4]
    w2 = result[5]
    w3 = result[6]
    theta1 = result[1]
    theta2 = result[2]
    theta3 = result[3]

    result = gradient_descent(X_train_mini, y_train_mini, w1, w2, w3, theta1, theta2, theta3, 0.00001, 0.00005, 5,N)
    output = predict_accuracy(X_test, y_test, theta1, theta2, theta3, w1, w2, w3)
    print(i, output)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: overflow encountered in exp


0 0.0630285714286
1 0.0586285714286
2 0.0546285714286
3 0.0547428571429
4 0.0579428571429
5 0.0584
6 0.0605142857143
7 0.0624
8 0.0658857142857
9 0.0672
10 0.0687428571429
11 0.0726285714286
12 0.076
13 0.0803428571429
14 0.0854285714286
15 0.0888571428571
16 0.0924571428571
17 0.0965714285714
18 0.0992571428571
19 0.102457142857
20 0.105428571429
21 0.108857142857
22 0.110742857143
23 0.114514285714
24 0.116914285714
25 0.12
26 0.123657142857
27 0.126285714286
28 0.130514285714
29 0.133942857143
30 0.136914285714
31 0.141314285714
32 0.145257142857
33 0.152171428571
34 0.157371428571
35 0.160171428571
36 0.164571428571
37 0.169028571429
38 0.172171428571
39 0.178057142857
40 0.182171428571
41 0.185942857143
42 0.188742857143
43 0.191371428571
44 0.1932
45 0.199485714286
46 0.203028571429
47 0.208742857143
48 0.214114285714
49 0.217771428571
50 0.220914285714
51 0.222742857143
52 0.2232
53 0.226228571429
54 0.226114285714
55 0.228228571429
56 0.232628571429
57 0.235028571429
58 0.23885